In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [2]:
# Load pre-trained model (weights)
#model_version = 'bert-large-cased'
model_version = 'bert-base-multilingual-cased'
model_version = '/virtualmachines/models/bert_multilingual_finetuned_6epochs'
model = BertForMaskedLM.from_pretrained(model_version)
model.eval()
cuda = torch.cuda.is_available()
if cuda:
    model = model.cuda(0)
cuda = False

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=model_version.endswith("uncased"))

def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def untokenize_batch(batch):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            if len(new_sent) == 0:
                continue
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

CLS = '[CLS]'
SEP = '[SEP]'
MASK = '[MASK]'
mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
sep_id = tokenizer.convert_tokens_to_ids([SEP])[0]
cls_id = tokenizer.convert_tokens_to_ids([CLS])[0]

/virtualmachines/models/bert_multilingual_finetuned_6epochs/vocab.txt


# Generations

In [3]:
def generate_step(out, gen_idx, temperature=None, top_k=0, sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """
    logits = out[:, gen_idx]
    if temperature is not None:
        logits = logits / temperature
    if top_k > 0:
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample:
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else:
        idx = torch.argmax(logits, dim=-1)
    return idx.tolist() if return_list else idx

In [ ]:
# Generation modes as functions
import math
import time
import random
from scipy.spatial.distance import cosine

def get_init_text(seed_text, max_len, batch_size = 1, rand_init=False):
    """ Get initial sentence by padding seed_text with either masks or random words to max_len """
    
    seed_text_copy = seed_text.copy()
    if len(seed_text) > max_len:
        raise Exception("Seed text is longer than max length")
    batch = []
    for j in range(batch_size):
        for i in range(len(seed_text)):
            sentence = [MASK] * (max_len + 2)                    
            sentence[0] = CLS
            sentence[1] = random.choice(["I", "You", "He", "She", "It", "We", "They", "My", "Our", "Your", "His", "Her", "Their"])
            #seed_text.append(sentence[1])
            sentence[-1] = SEP
            sentence[-2] = random.choice([".", "?", "!"])
            #seed_text.append(sentence[-2])
            sentence[random.randint(2, max_len-2)] = seed_text[i]
            sentence[random.randint(2, max_len-2)] = random.choice([s for s in seed_text if s != seed_text[i]])        
            batch.append(sentence)
    print(batch)
    return tokenize_batch(batch)

def parallel_sequential_generation(seed_text, max_len=15, top_k=0, temperature=None, max_iter=300, burnin=200,
                                   cuda=False, print_every=10, verbose=True):
    """ Generate for one random position at a timestep
    args:
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
    """
    try:
        batch = get_init_text(seed_text, max_len, batch_size)
    except Exception:
        return [],[],[]
    
    original = tokenizer.convert_tokens_to_ids(seed_text)
    
    batch_vectors = [None] * len(batch)
    variances = []
        
    for i in range(max_iter):
        kk = random.randint(1, max_len)
        complete = 0
        for ii in range(len(batch)):
            if batch[ii][kk] not in original:
                batch[ii][kk] = mask_id
            else:
                complete += 1
                
        if complete == len(batch):
            break
        
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)

        out = model(inp)
        
        topk = top_k if (i >= burnin) else 0
        idxs = generate_step(out, gen_idx=kk, top_k=topk, temperature=temperature, sample=(i<burnin))
        
        for ii in range(len(batch)):
            if batch[ii][kk] == mask_id:
                batch[ii][kk] = idxs[ii]
                #variances[ii] = np.sum(np.var(out.topk(5, dim=-1)[0].cpu().detach().numpy(),axis=1))
            batch_vectors[ii] = out[ii]
        #if i % 100 == 0:
        #    print(untokenize_batch(batch))
            
    for i in range(max_len):
        original_tokens = [None] * len(batch)
        for j in range(len(batch)):
            original_tokens[j] = batch[j][i]
            batch[j][i] = mask_id    
        out = model(torch.tensor(batch).cuda() if cuda else torch.tensor(batch))
        topk = out.topk(10000, dim=-1)[0].cpu().detach().numpy()
        print("topk shape is {0}".format(topk.shape))
        topk_at_i = topk[:,i,:]
        print("topk_at_i shape is {0}".format(topk_at_i.shape))
        topk_var_at_i = np.var(topk_at_i,axis=1)
        print("topk_var_at_i shape is {0}".format(topk_var_at_i.shape))
        variances.append(topk_var_at_i)
        for j in range(len(batch)):
            batch[j][i] = original_tokens[j]
            
    return batch_vectors, untokenize_batch(batch), np.transpose(variances, (1,0))

def parallel_generation(seed_text, max_len=15, top_k=0, temperature=None, max_iter=300, sample=True, 
                        cuda=False, print_every=10, verbose=True):
    """ Generate for all positions at a time step """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        for kk in range(max_len):
            idxs = generate_step(out, gen_idx=kk, top_k=top_k, temperature=temperature, sample=sample)
            for jj in range(batch_size):
                batch[jj][kk] = idxs[jj]
            
        if verbose and np.mod(ii, print_every) == 0:
            print("iter", ii+1, " ".join(tokenizer.convert_ids_to_tokens(batch[0])))
    
    return untokenize_batch(batch)
            
def sequential_generation(seed_text, batch_size=2, max_len=15, leed_out_len=15, 
                          top_k=0, temperature=None, sample=True, cuda=False):
    """ Generate one word at a time, in L->R order """
    batch = get_init_text(seed_text, max_len, batch_size)
    max_iter=50
    for iteration in range(max_iter):
        for ii in range(max_len):
            inp = [sent[:ii+1+leed_out_len]+[sep_id] for sent in batch]
            inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)

            out = model(inp)
            idxs = generate_step(out, gen_idx=ii+1, top_k=top_k, temperature=temperature, sample=sample)
            for jj in range(batch_size):
                if batch[jj][ii+1] == mask_id or batch[jj][ii+1] not in batch[0]:
                    batch[jj][ii+1] = idxs[jj]
    return untokenize_batch(batch)


def generate(n_samples, seed_text="[CLS]", batch_size=10, max_len=25, 
             sample=True, top_k=100, temperature=1.0, burnin=200, max_iter=500,
             cuda=False, print_every=1):
    # main generation function to call
    vectors = []
    sentences = []
    n_batches = math.ceil(n_samples / batch_size)
    start_time = time.time()
    for batch_n in range(n_batches):
        batch_vectors, batch_tokens, batch_variances = parallel_sequential_generation(seed_text, max_len=max_len, top_k=top_k,
                                               temperature=temperature, burnin=burnin, max_iter=max_iter, 
                                               cuda=cuda, verbose=False)
        
        #batch = sequential_generation(seed_text,
        #                              batch_size=batch_size, 
         #                             max_len=max_len, 
        #                              top_k=top_k,
        #                              temperature=temperature, 
        #                              leed_out_len=leed_out_len, 
        #                              sample=sample, cuda=cuda)
        #batch_tokens = parallel_generation(seed_text, 
        #                            max_len=max_len, 
        #                            top_k=top_k,
        #                            temperature=temperature, 
        #                            sample=sample, 
        #                            verbose=False,
        #                            cuda=cuda,
         #                           max_iter=max_iter)
        
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            start_time = time.time()
        
        sentences += batch_tokens
        vectors.append(batch_vectors)
    return sentences, vectors,batch_variances

In [ ]:
# Utility functions

def printer(sent, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent)[1:-1]
    print(" ".join(sent))
    
def read_sents(in_file, should_detokenize=False):
    sents = [sent.strip().split() for sent in open(in_file).readlines()]
    if should_detokenize:
        sents = [detokenize(sent) for sent in sents]
    return sents

def write_sents(out_file, sents, should_detokenize=False):
    with open(out_file, "w") as out_fh:
        for sent in sents:
            sent = detokenize(sent[1:-1]) if should_detokenize else sent
            out_fh.write("%s\n" % " ".join(sent))

In [ ]:
import itertools
cuda = True
n_samples = 15
batch_size = 5
min_len = 3
max_len = 10
top_k = 10000
temperature = 0.1
iter_per_sentence_length = 5

leed_out_len = 5 # max_len
burnin = 1000
# UNUSED ############
sample = True #######
####################
max_iter = 1000


# Choose the prefix context
#seed_text = "a little dog".split()
vocab = [w.strip().split(" ") for w in open("/virtualmachines/data/german/textbooks/vocab.txt", "r").readlines()]
vocab = list(itertools.chain.from_iterable(vocab))

sentences = []

for i in range(min_len, max_len):
    for j in range(iter_per_sentence_length):
        for k in range(1, len(vocab) - 1):
            seed_text = [vocab[k-1], vocab[k], vocab[k+1]]
            bert_sents, bert_vectors, bert_variances = generate(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                          sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
                          cuda=cuda)
    #out_file = "data/%s-len%d-burnin%d-topk%d-temp%.3f-samples%d-sample%s.txt" % (model_version.split("/")[-1], max_len, burnin, top_k, temp, n_samples, sample)
    #write_sents(out_file, bert_sents, should_detokenize=True)
            sentences += zip(bert_sents, bert_variances)
    
    #print("###############")
    #print(bert_sents[np.argmax(np.sum(bert_variances,axis=0))])
    #print(bert_sents)
    #print("###############")

#bert_vectors = [list(itertools.chain.from_iterable(bert_vectors))]
#best_sentences

sorted_sentences = sorted(sentences, key=lambda x: np.sum(x[1]) / x[1].shape[0])
#sorted_sentences = sorted(sentences, key=lambda x: len(x[0]))
sorted_sentences.reverse()
sorted_sentences

for i in sorted_sentences:
    joined = " ".join(i[0][1:-1])
    joined = joined.replace(" ,", ",")
    joined = joined.replace("  ", " ")
    joined = joined.replace(" ?", "?")
    joined = joined.replace(" .", ".")
    joined = joined.replace("  #", "")
    joined = joined.replace(" ##", "")
    #joined = joined.replace(" \"", "\"")
    #joined = joined.replace("\" ", "\"")
    print(joined)
    

In [4]:
import itertools
temperature = 1
top_k = 7

def tokenize_and_normalize_lengths(sentence):
    tokenized_sentence = tokenizer.tokenize(sentence)
    return tokenized_sentence

def mask_left_to_right(tokenized_sentence):
    masked = []
    for i in range(len(tokenized_sentence)):
        sentence = [CLS]
        for j in range(len(tokenized_sentence)):
            if i != j:
                sentence.append(tokenized_sentence[j])
            else:
                sentence.append(MASK)
        sentence.append(SEP)
        masked.append(sentence)
    return masked

with open("/virtualmachines/data/german/generated/sentences1.txt") as infile:
    sentences = infile.readlines()
    for sentence in sentences:
        tokenized_sentence = tokenize_and_normalize_lengths(sentence)
        masked_sentences = mask_left_to_right(tokenized_sentence)
        word_id_sentences = [tokenizer.convert_tokens_to_ids(masked_sentence) for masked_sentence in masked_sentences] 
        logits = model(torch.tensor(word_id_sentences).cuda()) / temperature
        #print(logits.size())
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        #print(kth_idx.size())
        # for every masked token position (points to the masked index AFTER CLS and SEP have been stripped)
        for mask_index in range(len(masked_sentences)):
            print(masked_sentences[mask_index])
            # get all possible options (topk) at this position (offset by one to skip suggestions for CLS)
            options = kth_idx[mask_index,mask_index+1].tolist()
            print(kth_vals[mask_index,mask_index+1].tolist())
            #print(options)
            # for every possible option
            for option_index in range(top_k):
                sentence = ""
                # reconstruct the sentence using the option at this position and the original (tokenized) text
                for word_index in range(len(tokenized_sentence)):
                    if word_index == mask_index:
                        sentence += tokenizer.convert_ids_to_tokens([options[option_index]])[0]
                    else:
                        sentence += tokenized_sentence[word_index]
                    sentence += " "
                print(sentence)
            print("####")

['[CLS]', '[MASK]', 'studier', '##t', 'derzeit', 'in', 'Deutschland', 'für', 'ihre', 'Promotion', '.', '[SEP]']
[10.238511085510254, 6.356626510620117, 5.558120250701904, 5.34294319152832, 5.054740905761719, 4.689211845397949, 4.356106758117676]
Sie studier ##t derzeit in Deutschland für ihre Promotion . 
sie studier ##t derzeit in Deutschland für ihre Promotion . 
Diese studier ##t derzeit in Deutschland für ihre Promotion . 
Er studier ##t derzeit in Deutschland für ihre Promotion . 
Es studier ##t derzeit in Deutschland für ihre Promotion . 
Die studier ##t derzeit in Deutschland für ihre Promotion . 
So studier ##t derzeit in Deutschland für ihre Promotion . 
####
['[CLS]', 'Sie', '[MASK]', '##t', 'derzeit', 'in', 'Deutschland', 'für', 'ihre', 'Promotion', '.', '[SEP]']
[11.515056610107422, 11.0024995803833, 10.320305824279785, 9.619972229003906, 9.57299518585205, 9.357550621032715, 9.174919128417969]
Sie ehr ##t derzeit in Deutschland für ihre Promotion . 
Sie sing ##t derzeit in 

[12.073419570922852, 11.289501190185547, 10.170231819152832, 9.354493141174316, 8.217129707336426, 8.054731369018555, 7.5787811279296875]
Seine eigene Einstellung zum ehe ##lichen Leben war etwas ganz anderes . 
Die eigene Einstellung zum ehe ##lichen Leben war etwas ganz anderes . 
seine eigene Einstellung zum ehe ##lichen Leben war etwas ganz anderes . 
Ihre eigene Einstellung zum ehe ##lichen Leben war etwas ganz anderes . 
Eine eigene Einstellung zum ehe ##lichen Leben war etwas ganz anderes . 
Meine eigene Einstellung zum ehe ##lichen Leben war etwas ganz anderes . 
die eigene Einstellung zum ehe ##lichen Leben war etwas ganz anderes . 
####
['[CLS]', 'Meine', '[MASK]', 'Einstellung', 'zum', 'ehe', '##lichen', 'Leben', 'war', 'etwas', 'ganz', 'anderes', '.', '[SEP]']
[9.150130271911621, 9.13869857788086, 8.510322570800781, 8.468283653259277, 8.418957710266113, 8.228107452392578, 7.440475940704346]
Meine politische Einstellung zum ehe ##lichen Leben war etwas ganz anderes . 
Meine 

[25.448400497436523, 15.769436836242676, 14.631673812866211, 14.347505569458008, 13.974884986877441, 13.348163604736328, 13.200603485107422]
Seine Stimme war sehr leis ##e und kon ##zent ##rierte sich auf mich . 
Seine Stimme war sehr leis ##e und ak ##zent ##rierte sich auf mich . 
Seine Stimme war sehr leis ##e und ex ##zent ##rierte sich auf mich . 
Seine Stimme war sehr leis ##e und pro ##zent ##rierte sich auf mich . 
Seine Stimme war sehr leis ##e und per ##zent ##rierte sich auf mich . 
Seine Stimme war sehr leis ##e und con ##zent ##rierte sich auf mich . 
Seine Stimme war sehr leis ##e und re ##zent ##rierte sich auf mich . 
####
['[CLS]', 'Seine', 'Stimme', 'war', 'sehr', 'leis', '##e', 'und', 'kon', '[MASK]', '##rierte', 'sich', 'auf', 'mich', '.', '[SEP]']
[20.23442840576172, 15.091876029968262, 14.466426849365234, 14.257826805114746, 14.0457181930542, 13.414436340332031, 13.374968528747559]
Seine Stimme war sehr leis ##e und kon ##zent ##rierte sich auf mich . 
Seine Stimm

[20.345664978027344, 14.593323707580566, 14.47823429107666, 12.939990997314453, 12.537447929382324, 12.363054275512695, 12.321840286254883]
Wer interes ##sierte sich dafür , Live - Action ##film ##e zu lesen ? 
Wer ##sierte ##sierte sich dafür , Live - Action ##film ##e zu lesen ? 
Wer arra ##sierte sich dafür , Live - Action ##film ##e zu lesen ? 
Wer ##zen ##sierte sich dafür , Live - Action ##film ##e zu lesen ? 
Wer ag ##sierte sich dafür , Live - Action ##film ##e zu lesen ? 
Wer ##ias ##sierte sich dafür , Live - Action ##film ##e zu lesen ? 
Wer sprach ##sierte sich dafür , Live - Action ##film ##e zu lesen ? 
####
['[CLS]', 'Wer', 'interes', '[MASK]', 'sich', 'dafür', ',', 'Live', '-', 'Action', '##film', '##e', 'zu', 'lesen', '?', '[SEP]']
[16.86980438232422, 11.912869453430176, 11.696863174438477, 11.616618156433105, 11.108061790466309, 10.713667869567871, 10.61435317993164]
Wer interes ##siert sich dafür , Live - Action ##film ##e zu lesen ? 
Wer interes ##sierte sich dafür 

[15.211395263671875, 14.144204139709473, 10.448612213134766, 9.858539581298828, 9.693138122558594, 9.481122016906738, 9.460691452026367]
Der Vater , ein Foto ##graf , besuchte die Schule . 
Sein Vater , ein Foto ##graf , besuchte die Schule . 
sein Vater , ein Foto ##graf , besuchte die Schule . 
der Vater , ein Foto ##graf , besuchte die Schule . 
Seine Vater , ein Foto ##graf , besuchte die Schule . 
Ein Vater , ein Foto ##graf , besuchte die Schule . 
Ihr Vater , ein Foto ##graf , besuchte die Schule . 
####
['[CLS]', 'Sein', '[MASK]', ',', 'ein', 'Foto', '##graf', ',', 'besuchte', 'die', 'Schule', '.', '[SEP]']
[14.053940773010254, 12.710169792175293, 10.211634635925293, 9.83474063873291, 9.456865310668945, 9.448945045471191, 8.672820091247559]
Sein Vater , ein Foto ##graf , besuchte die Schule . 
Sein Bruder , ein Foto ##graf , besuchte die Schule . 
Sein ältere , ein Foto ##graf , besuchte die Schule . 
Sein Onkel , ein Foto ##graf , besuchte die Schule . 
Sein Sohn , ein Foto ##

...

['[CLS]', '[MASK]', 'Mutter', 'war', 'einde', '##uti', '##g', 'den', 'T', '##r', '##änen', 'nahe', '.', '[SEP]']
[12.228461265563965, 11.64512825012207, 10.167863845825195, 8.751139640808105, 8.53752326965332, 7.458418369293213, 7.103490352630615]
Seine Mutter war einde ##uti ##g den T ##r ##änen nahe . 
Die Mutter war einde ##uti ##g den T ##r ##änen nahe . 
Ihre Mutter war einde ##uti ##g den T ##r ##änen nahe . 
Meine Mutter war einde ##uti ##g den T ##r ##änen nahe . 
seine Mutter war einde ##uti ##g den T ##r ##änen nahe . 
ihre Mutter war einde ##uti ##g den T ##r ##änen nahe . 
Eine Mutter war einde ##uti ##g den T ##r ##änen nahe . 
####
['[CLS]', 'Ihre', '[MASK]', 'war', 'einde', '##uti', '##g', 'den', 'T', '##r', '##änen', 'nahe', '.', '[SEP]']
[8.637323379516602, 8.059704780578613, 7.7899885177612305, 7.7361836433410645, 7.530044078826904, 7.514297962188721, 7.478877067565918]
Ihre Arbeit war einde ##uti ##g den T ##r ##änen nahe . 
Ihre Rolle war einde ##uti ##g den T ##r #

[10.122222900390625, 8.204361915588379, 8.068437576293945, 7.508093357086182, 6.285690784454346, 6.268129825592041, 5.802272796630859]
Der Foto ##graf sagt , dass er nie wieder ler ##nen wird . 
Der Foto ##graf sagt , wenn er nie wieder ler ##nen wird . 
Der Foto ##graf sagt , und er nie wieder ler ##nen wird . 
Der Foto ##graf sagt , wie er nie wieder ler ##nen wird . 
Der Foto ##graf sagt , als er nie wieder ler ##nen wird . 
Der Foto ##graf sagt , oder er nie wieder ler ##nen wird . 
Der Foto ##graf sagt , weil er nie wieder ler ##nen wird . 
####
['[CLS]', 'Der', 'Foto', '##graf', 'sagt', ',', 'dass', '[MASK]', 'nie', 'wieder', 'ler', '##nen', 'wird', '.', '[SEP]']
[10.404172897338867, 7.230412483215332, 6.82576322555542, 5.876416206359863, 5.842803955078125, 5.338085651397705, 5.1881537437438965]
Der Foto ##graf sagt , dass er nie wieder ler ##nen wird . 
Der Foto ##graf sagt , dass sie nie wieder ler ##nen wird . 
Der Foto ##graf sagt , dass es nie wieder ler ##nen wird . 
Der Fo

[13.601322174072266, 12.122462272644043, 11.573659896850586, 11.353782653808594, 10.813450813293457, 10.243767738342285, 10.029168128967285]
Wir hatten ein s ##ch ##öne ##s ru ##hig ##es Report ##ages ##sen und sprach ##en miteinander . 
Wir hatten ein s ##ch ##öne ##s ru ##hig ##es Mitt ##ages ##sen und sprach ##en miteinander . 
Wir hatten ein s ##ch ##öne ##s ru ##hig ##es W ##ages ##sen und sprach ##en miteinander . 
Wir hatten ein s ##ch ##öne ##s ru ##hig ##es Bin ##ages ##sen und sprach ##en miteinander . 
Wir hatten ein s ##ch ##öne ##s ru ##hig ##es Var ##ages ##sen und sprach ##en miteinander . 
Wir hatten ein s ##ch ##öne ##s ru ##hig ##es Vers ##ages ##sen und sprach ##en miteinander . 
Wir hatten ein s ##ch ##öne ##s ru ##hig ##es Lang ##ages ##sen und sprach ##en miteinander . 
####
['[CLS]', 'Wir', 'hatten', 'ein', 's', '##ch', '##öne', '##s', 'ru', '##hig', '##es', 'Mitt', '[MASK]', '##sen', 'und', 'sprach', '##en', 'miteinander', '.', '[SEP]']
[18.83060073852539, 15.29

['[CLS]', '[MASK]', '##vor', 'er', 'nach', 'Großbritannien', 'ging', ',', 'heiratete', 'er', 'zweimal', '.', '[SEP]']
[16.193546295166016, 13.263484954833984, 12.802851676940918, 11.24191665649414, 10.74508285522461, 10.690714836120605, 10.193411827087402]
Be ##vor er nach Großbritannien ging , heiratete er zweimal . 
be ##vor er nach Großbritannien ging , heiratete er zweimal . 
Wo ##vor er nach Großbritannien ging , heiratete er zweimal . 
##s ##vor er nach Großbritannien ging , heiratete er zweimal . 
##en ##vor er nach Großbritannien ging , heiratete er zweimal . 
##ingt ##vor er nach Großbritannien ging , heiratete er zweimal . 
da ##vor er nach Großbritannien ging , heiratete er zweimal . 
####
['[CLS]', 'Be', '[MASK]', 'er', 'nach', 'Großbritannien', 'ging', ',', 'heiratete', 'er', 'zweimal', '.', '[SEP]']
[14.74035930633545, 12.99008560180664, 11.388443946838379, 11.061312675476074, 10.651469230651855, 10.596399307250977, 10.58944034576416]
Be ##vor er nach Großbritannien ging 

['[CLS]', '[MASK]', 'Eltern', '.', '.', '.', 'Ja', ',', 'ihre', 'Eltern', 'waren', 'tot', '.', '[SEP]']
[11.308483123779297, 10.918964385986328, 9.047155380249023, 7.502802848815918, 7.444827556610107, 7.2131476402282715, 7.0937180519104]
Ihre Eltern . . . Ja , ihre Eltern waren tot . 
ihre Eltern . . . Ja , ihre Eltern waren tot . 
Die Eltern . . . Ja , ihre Eltern waren tot . 
die Eltern . . . Ja , ihre Eltern waren tot . 
Meine Eltern . . . Ja , ihre Eltern waren tot . 
seine Eltern . . . Ja , ihre Eltern waren tot . 
ihren Eltern . . . Ja , ihre Eltern waren tot . 
####
['[CLS]', 'Ihre', '[MASK]', '.', '.', '.', 'Ja', ',', 'ihre', 'Eltern', 'waren', 'tot', '.', '[SEP]']
[8.916996002197266, 8.168498039245605, 8.025176048278809, 7.9034929275512695, 7.635173797607422, 7.451501846313477, 7.296548843383789]
Ihre Kinder . . . Ja , ihre Eltern waren tot . 
Ihre Vater . . . Ja , ihre Eltern waren tot . 
Ihre Schwester . . . Ja , ihre Eltern waren tot . 
Ihre Eltern . . . Ja , ihre Eltern w

In [ ]:
batch

In [ ]:
sorted(range(10))

In [ ]:
max_len = 5
sent = [CLS] + ([MASK] * max_len)
sent[1] = "shoe"
sent[2] = "shoes"
sent[3] = "ran"
sent[4] = "run"
#sent[5] = "runs"
batch = tokenize_batch([sent])
inp = torch.tensor(batch).cuda() 
out = model(inp)
last = model.bert.embeddings.last_words_embeddings.cpu().detach().numpy() 
#model.bert.encoder.layer[0].attention.self.query.weight
#out[0,1] - out[0,2]
#print(out.shape)

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
y_pos = np.arange(last.shape[2])


plt.bar(y_pos, last[0,2] - last[0,1], align='center', alpha=0.5)

# Evaluation

In [ ]:
from nltk.translate import bleu_score as bleu

## Quality Measures

How similar are the generated sentences to the original training data (Toronto Book Corpus and Wikipedia dumps). We follow Yu et al., (2017) and compute the BLEU between the generations and the test sets of both corpora by treating the test set as the references for each generation. The tests sets are large; we subsample 5000 examples from each.

In [ ]:
def prepare_data(data_file, replacements={}, uncased=True):
    data = [d.strip().split() for d in open(data_file, 'r').readlines()]
    if uncased:
        data = [[t.lower() for t in sent] for sent in data]
        
    for k, v in replacements.items():
        data = [[t if t != k else v for t in sent] for sent in data]
 
    return data

def prepare_wiki(data_file, uncased=True):
    replacements = {"@@unknown@@": "[UNK]"}
    return prepare_data(data_file, replacements=replacements, uncased=uncased)

def prepare_tbc(data_file):        
    replacements = {"``": "\"", "\'\'": "\""}
    return prepare_data(data_file, replacements=replacements)

def corpus_bleu(generated, references):
    """ Compute similarity between two corpora as measured by
    comparing each sentence of `generated` against all sentences in `references` 
    
    args:
        - generated (List[List[str]]): list of sentences (split into tokens)
        - references (List[List[str]]): list of sentences (split into tokens)
        
    returns:
        - bleu (float)
    """    
    return bleu.corpus_bleu([references for _ in range(len(generated))], generated)

In [ ]:
wiki103_file = 'data/wiki103.5k.txt'
tbc_file = 'data/tbc.5k.txt'

wiki_data = prepare_wiki(wiki103_file)
tbc_data = prepare_tbc(tbc_file)
#sents = [detokenize(sent) for sent in sents]

In [ ]:
print("BERT-TBC BLEU: %.2f" % (100 * corpus_bleu(bert_sents, tbc_data)))
print("BERT-Wiki103 BLEU: %.2f" % (100 * corpus_bleu(bert_sents, wiki_data)))
print("BERT-{TBC + Wiki103} BLEU: %.2f" % (100 * corpus_bleu(bert_sents, tbc_data[:2500] + wiki_data[:2500])))

## Comparing to existing models

The OpenAI Generative Pretraining Transformer is another pretrained model successfully used for transfer learning. Since the model is a unidirectional language model, we can straightforwardly generate from the model. See [this repo](https://github.com/huggingface/pytorch-openai-transformer-lm) by Thomas Wolf at Huggingface for instructions for setting up the model.

In [ ]:
import os
import sys
sys.path.insert(1, os.path.join(".", "pytorch-openai-transformer-lm"))

from model_pytorch import LMModel, load_openai_pretrained_model, DEFAULT_CONFIG
from text_utils import TextEncoder

def load_openai_gpt(n_special=1, n_ctx=512):
    text_encoder = TextEncoder("/virtualmachines/models/pytorch-openai-transformer-lm/encoder_bpe_40000.json", 
                               "/virtualmachines/models/pytorch-openai-transformer-lm/vocab_40000.bpe")
    encoder = text_encoder.encoder
    n_vocab = len(text_encoder.encoder)
    vocab = n_vocab + n_special + n_ctx

    args = DEFAULT_CONFIG
    lm_model = LMModel(args, vocab, n_ctx, return_probs=True)
    load_openai_pretrained_model(lm_model.transformer, n_ctx=n_ctx, n_special=n_special,
                                 path="/virtualmachines/models/pytorch-openai-transformer-lm/",
                                 path_names="pytorch-openai-transformer-lm/")
    #lm_model.to(device)
    lm_model.return_probs = False
    lm_model.eval()
    return lm_model, text_encoder

def make_batch(X, n_vocab, n_special, batch_size):
    X = np.array(X)
    assert X.ndim in [1, 2]
    if X.ndim == 1:
        X = np.expand_dims(X, axis=0)
    pos_enc = np.arange(n_vocab + n_special, n_vocab + n_special + X.shape[-1])
    pos_enc = np.tile(pos_enc, (batch_size, 1)) #np.expand_dims(pos_enc, axis=0)
    batch = np.stack([X, pos_enc], axis=-1)
    batch = torch.tensor(batch, dtype=torch.long)#.to(device)
    return batch

def append_batch(X, next_idx):
    next_pos = X[:, -1:, 1] + 1
    next_x = torch.cat((next_idx, next_pos), -1).unsqueeze(1)
    return torch.cat((X, next_x), 1)

def _generate_sentence_openai(model, text_encoder, seed_text, batch_size=10, gen_len=20, 
                             topk=100, sample=True, n_special=0):
    n_vocab = len(text_encoder.encoder)
    #X = np.random.randint(n_vocab, size=(batch_size, 1)).tolist()
    #sents = [[text_encoder.decoder[X[i][0]]].replace('</w>', '') for i in range(batch_size)]
    X = [[n_vocab - 1] for _ in range(batch_size)]
    sents = [[] for _ in range(batch_size)]
    if seed_text:
        seed_ids = text_encoder.encode([seed_text,])
        X = [X[i] + seed_ids[0] for i in range(batch_size)]
        sents = [[seed_text] for _ in range(batch_size)]
    XMB = make_batch(X, n_vocab, n_special, batch_size=batch_size)


    for step_n in range(gen_len):
        out = model(XMB) + model.pos_emb_mask
        next_idxs = generate_step(out, gen_idx=step_n, top_k=topk, sample=sample, return_list=False)
        idxs = next_idxs.tolist()
        for i in range(batch_size):
            next_token = idxs[i]
            if next_token == n_vocab:
                next_token = "<EOS>"
            else:
                next_token = text_encoder.decoder[next_token].replace('</w>', '')
            sents[i].append(next_token)
        XMB = append_batch(XMB, next_idxs.unsqueeze(-1))
        
    return [[tok for tok in sent if tok != '\n'] for sent in sents]

def generate_openai(model, text_encoder, n_samples, seed_text, 
                    batch_size=10, gen_len=20, 
                    topk=100, temperature=temperature, sample=sample,
                    n_special=0, print_every=1):
    sents = []
    start_time = time.time()
    n_batches = math.ceil(n_samples / batch_size)
    for batch_n in range(n_batches):
        batch_sents = _generate_sentence_openai(model, text_encoder, seed_text,
                                                batch_size=batch_size, gen_len=gen_len, 
                                                topk=topk, sample=sample,
                                                n_special=n_special)
        sents += batch_sents
        if (batch_n + 1) % print_every == 0:
            print("Generated batch %d of %d in %.3fs" % (batch_n + 1, n_batches, time.time() - start_time))
            start_time = time.time()
    return sents

In [ ]:
gpt_model, gpt_text_encoder = load_openai_gpt(n_special=1)

In [ ]:
n_samples = 10
batch_size = 5
max_len = 5
top_k = 50
temperature = 0.1

sample = False

openai_sents = generate_openai(gpt_model, gpt_text_encoder, seed_text="dog", 
                               n_samples=n_samples, batch_size=batch_size, gen_len=max_len,
                               topk=top_k, temperature=temperature, sample=sample,
                               n_special=1, print_every=1)
openai_sents

In [ ]:
printer(openai_sents[9], should_detokenize=False)

In [ ]:
print("GPT-TBC BLEU: %.2f" % (100 * corpus_bleu(openai_sents, tbc_data)))
print("GPT-Wiki103 BLEU: %.2f" % (100 * corpus_bleu(openai_sents, wiki_data)))
print("GPT-{TBC + Wiki103} BLEU: %.2f" % (100 * corpus_bleu(openai_sents, tbc_data[:2500] + wiki_data[:2500])))

## Diversity Measures

Self-BLEU: treat each sentence as a hypothesis and treat rest of corpus as reference. Lower is better.

In [ ]:
from collections import Counter
from nltk.util import ngrams

def self_bleu(sents):
    return bleu.corpus_bleu([[s for (j, s) in enumerate(sents) if j != i] for i in range(len(sents))], sents)

def get_ngram_counts(sents, max_n=4):
    size2count = {}
    for i in range(1, max_n + 1):
        size2count[i] = Counter([n for sent in sents for n in ngrams(sent, i)])
    return size2count

def ref_unique_ngrams(preds, refs, max_n=4):
    # get # of *distinct* pred ngrams that don't appear in ref
    pct_unique = {}
    pred_ngrams = get_ngram_counts(preds, max_n)
    ref_ngrams = get_ngram_counts(refs, max_n)
    for i in range(1, max_n + 1):
        pred_ngram_counts = set(pred_ngrams[i].keys())
        total = sum(pred_ngrams[i].values())
        ref_ngram_counts = set(ref_ngrams[i].keys())
        pct_unique[i] = len(pred_ngram_counts.difference(ref_ngram_counts)) / total
    return pct_unique
        
def self_unique_ngrams(preds, max_n=4):
    # get # of pred ngrams with count 1
    pct_unique = {}
    pred_ngrams = get_ngram_counts(preds, max_n)
    for i in range(1, max_n + 1):
        n_unique = len([k for k, v in pred_ngrams[i].items() if v == 1])
        total = sum(pred_ngrams[i].values())
        pct_unique[i] = n_unique / total
    return pct_unique

In [ ]:
print("BERT self-BLEU: %.2f" % (100 * self_bleu(bert_sents)))
print("OpenAI self-BLEU: %.2f" % (100 * self_bleu(openai_sents)))

In [ ]:
max_n = 4

pct_uniques = ref_unique_ngrams(bert_sents, wiki_data, max_n)
for i in range(1, max_n + 1):
    print("BERT unique %d-grams relative to Wiki: %.2f" % (i, 100 * pct_uniques[i]))
pct_uniques = ref_unique_ngrams(bert_sents, tbc_data, max_n)
for i in range(1, max_n + 1):
    print("BERT unique %d-grams relative to TBC: %.2f" % (i, 100 * pct_uniques[i]))
pct_uniques = self_unique_ngrams(bert_sents, max_n)
for i in range(1, max_n + 1):
    print("BERT unique %d-grams relative to self: %.2f" % (i, 100 * pct_uniques[i]))

In [ ]:
pct_uniques = ref_unique_ngrams(openai_sents, wiki_data, max_n)
for i in range(1, max_n + 1):
    print("GPT unique %d-grams relative to Wiki: %.2f" % (i, 100 * pct_uniques[i]))
pct_uniques = ref_unique_ngrams(openai_sents, tbc_data, max_n)
for i in range(1, max_n + 1):
    print("GPT unique %d-grams relative to TBC: %.2f" % (i, 100 * pct_uniques[i]))
pct_uniques = self_unique_ngrams(openai_sents, max_n)
for i in range(1, max_n + 1):
    print("GPT unique %d-grams relative to self: %.2f" % (i, 100 * pct_uniques[i]))